In [13]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv, splitrepeat_mcn
# from splitrepeat1 import splitrepeat_cv, splitrepeat_mc

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features = ['ADAS_Q4', 'e_organ_cg', 'e_memory_cg', 'ADAS_Q1', 'e_plan_cg', 'tmab_time', 'AVDEL30MIN', 'faq1', 'faq10', 'moca_recall', 'faq2', 'LDELTOTAL', 'e_lang_pt', 'e_memory_pt', 'faq9', 'e_lang_cg', 'e_divatt_cg']

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({0:1,1:0,2:0},inplace=True)

In [2]:
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

# RF Classifier

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 3.22 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.937037,0.878977,0.910969,0.908007,0.878977,0.937037,0.910969,0.908007,0.878977,0.937037,0.910969,0.908007
std,0.035874,0.033644,0.021100,0.020716,0.033644,0.035874,0.021100,0.020716,0.033644,0.035874,0.021100,0.020716
min,0.888889,0.829545,0.877551,0.876263,0.829545,0.888889,0.877551,0.876263,0.829545,0.888889,0.877551,0.876263
25%,0.907407,0.860795,0.896684,0.890730,0.860795,0.907407,0.896684,0.890730,0.860795,0.907407,0.896684,0.890730
50%,0.930556,0.869318,0.910714,0.907934,0.869318,0.930556,0.910714,0.907934,0.869318,0.930556,0.910714,0.907934
75%,0.965278,0.892045,0.933673,0.927452,0.892045,0.965278,0.933673,0.927452,0.892045,0.965278,0.933673,0.927452
max,0.990741,0.943182,0.943878,0.938552,0.943182,0.990741,0.943878,0.938552,0.943182,0.990741,0.943878,0.938552


## Random Search

In [5]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [6]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   10.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   50.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  6.0min finished


RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(max_features=0.5,
                                                    random_state=555),
                   n_iter=400, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['sqrt', 'log2', 0.3,
                                                         0.5, 0.7, 0.9],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                        

In [7]:
rf_search_rand.best_params_

{'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 0.3,
 'max_depth': 20,
 'bootstrap': True}

In [8]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.925926,0.849747,0.891723,0.887837,0.849747,0.925926,0.891723,0.887837,0.849747,0.925926,0.891723,0.887837
std,0.024498,0.038860,0.011345,0.012879,0.038860,0.024498,0.011345,0.012879,0.038860,0.024498,0.011345,0.012879
min,0.898148,0.795455,0.882653,0.874579,0.795455,0.898148,0.882653,0.874579,0.795455,0.898148,0.882653,0.874579
25%,0.898148,0.806818,0.882653,0.880261,0.806818,0.898148,0.882653,0.880261,0.806818,0.898148,0.882653,0.880261
50%,0.925926,0.863636,0.887755,0.880892,0.863636,0.925926,0.887755,0.880892,0.863636,0.925926,0.887755,0.880892
75%,0.953704,0.875000,0.903061,0.901515,0.875000,0.953704,0.903061,0.901515,0.875000,0.953704,0.903061,0.901515
max,0.953704,0.886364,0.908163,0.906145,0.886364,0.953704,0.908163,0.906145,0.886364,0.953704,0.908163,0.906145


## Grid Search

In [9]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [700, 800, 900],
               'max_features': [.2, .3, .4],
               'max_depth': [10, 20, 30],
               'min_samples_split': [8, 9, 10],
               'min_samples_leaf': [1, 2, 3],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 486 candidates, totalling 1458 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   27.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 1458 out of 1458 | elapsed:  5.1min finished


GridSearchCV(cv=3,
             estimator=RandomForestClassifier(max_features=0.5,
                                              random_state=555),
             n_jobs=-1,
             param_grid={'bootstrap': [False, True], 'max_depth': [10, 20, 30],
                         'max_features': [0.2, 0.3, 0.4],
                         'min_samples_leaf': [1, 2, 3],
                         'min_samples_split': [8, 9, 10],
                         'n_estimators': [700, 800, 900]},
             verbose=2)

In [10]:
rf_search_grid.best_params_

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 0.3,
 'min_samples_leaf': 1,
 'min_samples_split': 10,
 'n_estimators': 800}

In [11]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.925926,0.849747,0.891723,0.887837,0.849747,0.925926,0.891723,0.887837,0.849747,0.925926,0.891723,0.887837
std,0.024498,0.038860,0.011345,0.012879,0.038860,0.024498,0.011345,0.012879,0.038860,0.024498,0.011345,0.012879
min,0.898148,0.795455,0.882653,0.874579,0.795455,0.898148,0.882653,0.874579,0.795455,0.898148,0.882653,0.874579
25%,0.898148,0.806818,0.882653,0.880261,0.806818,0.898148,0.882653,0.880261,0.806818,0.898148,0.882653,0.880261
50%,0.925926,0.863636,0.887755,0.880892,0.863636,0.925926,0.887755,0.880892,0.863636,0.925926,0.887755,0.880892
75%,0.953704,0.875000,0.903061,0.901515,0.875000,0.953704,0.903061,0.901515,0.875000,0.953704,0.903061,0.901515
max,0.953704,0.886364,0.908163,0.906145,0.886364,0.953704,0.908163,0.906145,0.886364,0.953704,0.908163,0.906145


In [12]:
rf = RandomForestClassifier(n_estimators=800, max_features=.3, max_depth = 20, min_samples_split = 10, min_samples_leaf = 1, bootstrap=True, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 22 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.943981,0.888636,0.919133,0.916309,0.888636,0.943981,0.919133,0.916309,0.888636,0.943981,0.919133,0.916309
std,0.028889,0.035093,0.020891,0.021228,0.035093,0.028889,0.020891,0.021228,0.035093,0.028889,0.020891,0.021228
min,0.916667,0.852273,0.892857,0.890152,0.852273,0.916667,0.892857,0.890152,0.852273,0.916667,0.892857,0.890152
25%,0.925926,0.863636,0.897959,0.894781,0.863636,0.925926,0.897959,0.894781,0.863636,0.925926,0.897959,0.894781
50%,0.930556,0.869318,0.923469,0.920770,0.869318,0.930556,0.923469,0.920770,0.869318,0.930556,0.923469,0.920770
75%,0.962963,0.900568,0.938776,0.935553,0.900568,0.962963,0.938776,0.935553,0.900568,0.962963,0.938776,0.935553
max,0.990741,0.954545,0.943878,0.940236,0.954545,0.990741,0.943878,0.940236,0.954545,0.990741,0.943878,0.940236


In [14]:
# All samples

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(10)),repeats=list(int(x)*42+42 for x in range(10)), initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 2min 12s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.927814,0.858475,0.896505,0.893144,0.858475,0.927814,0.896505,0.893144,0.858475,0.927814,0.896505,0.893144
std,0.018231,0.028565,0.012943,0.013687,0.028565,0.018231,0.012943,0.013687,0.028565,0.018231,0.012943,0.013687
min,0.893023,0.813559,0.875000,0.870070,0.813559,0.893023,0.875000,0.870070,0.813559,0.893023,0.875000,0.870070
25%,0.911628,0.830508,0.887755,0.885192,0.830508,0.911628,0.887755,0.885192,0.830508,0.911628,0.887755,0.885192
50%,0.925581,0.858757,0.894133,0.891092,0.858757,0.925581,0.894133,0.891092,0.858757,0.925581,0.894133,0.891092
75%,0.944186,0.875706,0.903061,0.899645,0.875706,0.944186,0.903061,0.899645,0.875706,0.944186,0.903061,0.899645
max,0.958140,0.920904,0.928571,0.925397,0.920904,0.958140,0.928571,0.925397,0.920904,0.958140,0.928571,0.925397


# Outputs

In [15]:
### Save outputs ###
q = 'CDR0_BorutaSHAP_2'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)